In [1]:
import pandas as pd
import scipy.stats
from collections import Counter
GEST = 'out_data/cor_full_GEST_DIABETES1.txt'
HYPT = 'out_data/cor_full_I9_HYPTENSPREG1.txt'
PRET = 'out_data/cor_full_O15_PRETERM1.txt'
FINN_UK = 'data/finn_uk_codes.csv'
PRIOR_LIST = ['2']

In [2]:
fu = pd.read_csv(FINN_UK)
fu = fu
fu.index = fu['0']+ '___' +fu['UK_code']
fu['full_name'] = 'FG: ' + fu['finn_pheno'] + ' & \nUK: ' + fu['UK_pheno']
fu = fu.loc[:,['full_name', 'priority']]
fu

,full_name,priority
E4_THYROID___C73,FG: Disorders of the thyroid gland & \nUK: Mal...,*
C3_THYROID_GLAND___C73,FG: Malignant neoplasm of thyroid gland & \nUK...,2
C3_THYROID_GLAND_EXALLC___C73,FG: Malignant neoplasm of thyroid gland (all c...,1
E4_THYTOXNOD___C73,FG: Thyrotoxicosis with toxic single thyroid n...,0
I9_RHEUVALV___I35,FG: Rheumatic valve diseases & \nUK: Nonrheuma...,*
...,...,...
L12_LUPUS___M32,FG: Lupus erythematosus & \nUK: Systemic lupus...,2
M13_SLE___M32,FG: Systemic lupus erythematosus & \nUK: Syste...,2
SLE_NOS___M32,"FG: Systemic lupus erythematosus, unspecified ...",2
SLE_OTH___M32,FG: Other forms of systemic lupus erythematosu...,1


In [3]:
ph_map = {'I9_HYPTENSPREG':'HP', 'GEST_DIABETES':'GD', 'O15_PRETERM':'PTB'}

In [4]:
def return_df(FILE):
    data = ''
    with open(FILE, 'r') as in_f:
        data = in_f.read().replace('\n--', '').split('\n')
    keys = list(map(lambda x: x.split('/')[0].replace('analysis_n_', ''), data[::2]))
    values = map(lambda x: x.split(' '), data[1::2])
    traits = map(lambda x: x.split('/')[-1].replace('1.cors', ''), data[::2])
    data_dict = {k: v for k, v in zip(keys, values)}
    for k, v in zip(keys, traits):
        try:
            data_dict[k].append(v)
        except KeyError:
            print(k)
    return pd.DataFrame(data_dict, index=['Cor_All', 'cor', 'cor_std', 'trait']).T


def count_sign(data):
    data['grep_stat'] = data.cor.astype(float)/data.cor_std.astype(float)
    data['pval'] = scipy.stats.norm.sf(abs(data['grep_stat']))*2
    data['p2'] = data.index
    PVAL_CUTOFF = 0.05/data.shape[0]
    data['significant'] = ''
    data.loc[data.pval<PVAL_CUTOFF, ['significant']] = '*'
    return data

In [5]:
# g, h = list(gest_data.index), list(hypt_data.index)
# len(g), len(h), len(set(g)), len(set(h)), len(set(g) & set(h))

In [6]:
gest_data  = return_df(GEST)
hypt_data = return_df(HYPT)
pret_data = return_df(PRET)
gest_data  = count_sign(gest_data)
hypt_data = count_sign(hypt_data)
pret_data = count_sign(pret_data)
display(gest_data)
display(hypt_data)
display(pret_data)

,Cor_All,cor,cor_std,trait,grep_stat,pval,p2,significant
AB1_BACT_INTEST_OTH___A04,Cor_All,0.434338,0.447854,GEST_DIABETES,0.969821,0.332136,AB1_BACT_INTEST_OTH___A04,
AB1_CANDIDIASIS___B37,Cor_All,-0.330457,0.529697,GEST_DIABETES,-0.623860,0.532719,AB1_CANDIDIASIS___B37,
AB1_GASTROENTERITIS_NOS___A09,Cor_All,1.082440,0.836931,GEST_DIABETES,1.293344,0.195892,AB1_GASTROENTERITIS_NOS___A09,
AB1_OTHER_SEPSIS___A41,Cor_All,0.306278,0.729840,GEST_DIABETES,0.419651,0.674741,AB1_OTHER_SEPSIS___A41,
AB1_SEPSIS___A41,Cor_All,0.227972,1.581424,GEST_DIABETES,0.144156,0.885377,AB1_SEPSIS___A41,
...,...,...,...,...,...,...,...,...
Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,Cor_All,0.245154,0.342948,GEST_DIABETES,0.714843,0.474706,Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,
Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA_REPOR_DIAGNOSIS___Z01,Cor_All,0.788314,0.305386,GEST_DIABETES,2.581369,0.009841,Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA...,
Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_PROCED_NOT_CARRIED___Z53,Cor_All,0.663892,0.433588,GEST_DIABETES,1.531159,0.125730,Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_P...,
Z21_PROCED_PURPO_OTH_REMED_HEALTH_STATE___Z41,Cor_All,0.411307,0.389700,GEST_DIABETES,1.055445,0.291222,Z21_PROCED_PURPO_OTH_REMED_HEALTH_STATE___Z41,


,Cor_All,cor,cor_std,trait,grep_stat,pval,p2,significant
AB1_BACT_INTEST_OTH___A04,Cor_All,-0.303086,0.332959,I9_HYPTENSPREG,-0.910280,0.362675,AB1_BACT_INTEST_OTH___A04,
AB1_CANDIDIASIS___B37,Cor_All,0.232431,0.406713,I9_HYPTENSPREG,0.571487,0.567670,AB1_CANDIDIASIS___B37,
AB1_GASTROENTERITIS_NOS___A09,Cor_All,0.244028,0.282443,I9_HYPTENSPREG,0.863990,0.387593,AB1_GASTROENTERITIS_NOS___A09,
AB1_OTHER_SEPSIS___A41,Cor_All,0.114207,0.325601,I9_HYPTENSPREG,0.350758,0.725770,AB1_OTHER_SEPSIS___A41,
AB1_SEPSIS___A41,Cor_All,0.094639,0.364655,I9_HYPTENSPREG,0.259530,0.795226,AB1_SEPSIS___A41,
...,...,...,...,...,...,...,...,...
Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI___Z03,Cor_All,0.373732,0.194132,I9_HYPTENSPREG,1.925144,0.054211,Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI_...,
Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,Cor_All,0.135456,0.201727,I9_HYPTENSPREG,0.671482,0.501914,Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,
Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA_REPOR_DIAGNOSIS___Z01,Cor_All,0.361813,0.170499,I9_HYPTENSPREG,2.122083,0.033831,Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA...,
Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_PROCED_NOT_CARRIED___Z53,Cor_All,0.487251,0.360603,I9_HYPTENSPREG,1.351212,0.176628,Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_P...,


,Cor_All,cor,cor_std,trait,grep_stat,pval,p2,significant
AB1_BACT_INTEST_OTH___A04,Cor_All,-0.271273,0.556522,O15_PRETERM,-0.487443,0.625944,AB1_BACT_INTEST_OTH___A04,
AB1_CANDIDIASIS___B37,Cor_All,0.500804,0.634897,O15_PRETERM,0.788796,0.430231,AB1_CANDIDIASIS___B37,
AB1_GASTROENTERITIS_NOS___A09,Cor_All,0.354631,0.561824,O15_PRETERM,0.631214,0.527901,AB1_GASTROENTERITIS_NOS___A09,
AB1_OTHER_SEPSIS___A41,Cor_All,0.484202,0.642383,O15_PRETERM,0.753759,0.450994,AB1_OTHER_SEPSIS___A41,
AB1_SEPSIS___A41,Cor_All,0.457686,0.738186,O15_PRETERM,0.620014,0.535248,AB1_SEPSIS___A41,
...,...,...,...,...,...,...,...,...
Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI___Z03,Cor_All,1.102863,0.678434,O15_PRETERM,1.625601,0.104035,Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI_...,
Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,Cor_All,0.255791,0.476292,O15_PRETERM,0.537047,0.591235,Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,
Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA_REPOR_DIAGNOSIS___Z01,Cor_All,0.749449,0.500880,O15_PRETERM,1.496265,0.134585,Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA...,
Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_PROCED_NOT_CARRIED___Z53,Cor_All,-0.213836,0.617483,O15_PRETERM,-0.346303,0.729115,Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_P...,


In [7]:
concatted_data = pd.concat([gest_data, hypt_data, pret_data])
concatted_data

,Cor_All,cor,cor_std,trait,grep_stat,pval,p2,significant
AB1_BACT_INTEST_OTH___A04,Cor_All,0.434338,0.447854,GEST_DIABETES,0.969821,0.332136,AB1_BACT_INTEST_OTH___A04,
AB1_CANDIDIASIS___B37,Cor_All,-0.330457,0.529697,GEST_DIABETES,-0.623860,0.532719,AB1_CANDIDIASIS___B37,
AB1_GASTROENTERITIS_NOS___A09,Cor_All,1.082440,0.836931,GEST_DIABETES,1.293344,0.195892,AB1_GASTROENTERITIS_NOS___A09,
AB1_OTHER_SEPSIS___A41,Cor_All,0.306278,0.729840,GEST_DIABETES,0.419651,0.674741,AB1_OTHER_SEPSIS___A41,
AB1_SEPSIS___A41,Cor_All,0.227972,1.581424,GEST_DIABETES,0.144156,0.885377,AB1_SEPSIS___A41,
...,...,...,...,...,...,...,...,...
Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI___Z03,Cor_All,1.102863,0.678434,O15_PRETERM,1.625601,0.104035,Z21_MEDICAL_OBSER_EVALU_SUSPECTED_DISEA_CONDI_...,
Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,Cor_All,0.255791,0.476292,O15_PRETERM,0.537047,0.591235,Z21_OTHER_ORTHOPA_FOLLOW__CARE___Z47,
Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA_REPOR_DIAGNOSIS___Z01,Cor_All,0.749449,0.500880,O15_PRETERM,1.496265,0.134585,Z21_OTHER_SPECIAL_EXAM_INVES_PERSONS_WO_COMPLA...,
Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_PROCED_NOT_CARRIED___Z53,Cor_All,-0.213836,0.617483,O15_PRETERM,-0.346303,0.729115,Z21_PERSONS_ENCOUNTERI_HEALTH_SERVI_SPECIFIC_P...,


In [8]:
filtered_concatted_data = concatted_data[concatted_data.significant=='*']
cd_final = concatted_data[concatted_data.p2.isin(set(filtered_concatted_data.index))]
cd_final

,Cor_All,cor,cor_std,trait,grep_stat,pval,p2,significant
E4_DIABETES___E14,Cor_All,0.930488,0.164872,GEST_DIABETES,5.643699,1.664346e-08,E4_DIABETES___E14,*
E4_OBESITY___E66,Cor_All,0.529482,0.153193,GEST_DIABETES,3.456307,5.476317e-04,E4_OBESITY___E66,
I9_HYPTENSESS___I10,Cor_All,0.500904,0.141347,GEST_DIABETES,3.543789,3.944202e-04,I9_HYPTENSESS___I10,
I9_IHD___I25,Cor_All,0.247124,0.091702,GEST_DIABETES,2.694859,7.041833e-03,I9_IHD___I25,
O15_MATERN_HYPERT_UNS___O16,Cor_All,0.107861,0.474805,GEST_DIABETES,0.227169,8.202923e-01,O15_MATERN_HYPERT_UNS___O16,
O15_PRETERM___O60,Cor_All,-0.065977,0.465984,GEST_DIABETES,-0.141586,8.874067e-01,O15_PRETERM___O60,
R18_PAIN_THROAT_CHEST___R07,Cor_All,0.410649,0.114888,GEST_DIABETES,3.574342,3.511098e-04,R18_PAIN_THROAT_CHEST___R07,
E4_DIABETES___E14,Cor_All,0.420760,0.098911,I9_HYPTENSPREG,4.253925,2.100555e-05,E4_DIABETES___E14,*
E4_OBESITY___E66,Cor_All,0.374283,0.092909,I9_HYPTENSPREG,4.028490,5.613619e-05,E4_OBESITY___E66,*
I9_HYPTENSESS___I10,Cor_All,0.699877,0.101525,I9_HYPTENSPREG,6.893642,5.438180e-12,I9_HYPTENSESS___I10,*


In [9]:
final = pd.merge(cd_final, fu,  how="left", left_index=True, right_index=True)
final.loc[final.p2== 'I9_HYPTENSESS___I10', ['priority']]='2'
final = final[final.index!='O15_PRETERM___O60']

col_rename = {"cor": "rg", "cor_std": "se", "trait": "p1", "grep_stat": "z", "pval": "p", "p2": "p2", "priority":"priority"}
final = final.rename(columns=col_rename)

final.p1 = final.p1.apply(lambda x: ph_map[x])
final.p2 = final.full_name


print(final.shape)
final

(18, 10)


,Cor_All,rg,se,p1,z,p,p2,significant,full_name,priority
E4_DIABETES___E14,Cor_All,0.930488,0.164872,GD,5.643699,1.664346e-08,FG: Diabetes mellitus & \nUK: Unspecified diab...,*,FG: Diabetes mellitus & \nUK: Unspecified diab...,2
E4_DIABETES___E14,Cor_All,0.420760,0.098911,HP,4.253925,2.100555e-05,FG: Diabetes mellitus & \nUK: Unspecified diab...,*,FG: Diabetes mellitus & \nUK: Unspecified diab...,2
E4_DIABETES___E14,Cor_All,0.300049,0.316877,PTB,0.946894,3.436927e-01,FG: Diabetes mellitus & \nUK: Unspecified diab...,,FG: Diabetes mellitus & \nUK: Unspecified diab...,2
E4_OBESITY___E66,Cor_All,0.529482,0.153193,GD,3.456307,5.476317e-04,FG: Obesity & \nUK: Obesity,,FG: Obesity & \nUK: Obesity,2
E4_OBESITY___E66,Cor_All,0.374283,0.092909,HP,4.028490,5.613619e-05,FG: Obesity & \nUK: Obesity,*,FG: Obesity & \nUK: Obesity,2
E4_OBESITY___E66,Cor_All,0.175672,0.260471,PTB,0.674440,5.000318e-01,FG: Obesity & \nUK: Obesity,,FG: Obesity & \nUK: Obesity,2
I9_HYPTENSESS___I10,Cor_All,0.500904,0.141347,GD,3.543789,3.944202e-04,"FG: Hypertension, essential & \nUK: Essential ...",,"FG: Hypertension, essential & \nUK: Essential ...",2
I9_HYPTENSESS___I10,Cor_All,0.699877,0.101525,HP,6.893642,5.438180e-12,"FG: Hypertension, essential & \nUK: Essential ...",*,"FG: Hypertension, essential & \nUK: Essential ...",2
I9_HYPTENSESS___I10,Cor_All,0.319135,0.311526,PTB,1.024425,3.056346e-01,"FG: Hypertension, essential & \nUK: Essential ...",,"FG: Hypertension, essential & \nUK: Essential ...",2
I9_IHD___I25,Cor_All,0.247124,0.091702,GD,2.694859,7.041833e-03,"FG: Ischaemic heart disease, wide definition &...",,"FG: Ischaemic heart disease, wide definition &...",2


In [10]:
data_to_save = final.loc[:,['p1', 'p2', 'rg', 'se', 'z', 'p', 'priority', 'significant']]
data_to_save = data_to_save[data_to_save.priority.astype(str).isin(PRIOR_LIST)]
data_to_save.to_csv('./out_data/meta_feature.csv', index=False)
data_to_save

,p1,p2,rg,se,z,p,priority,significant
E4_DIABETES___E14,GD,FG: Diabetes mellitus & \nUK: Unspecified diab...,0.930488,0.164872,5.643699,1.664346e-08,2,*
E4_DIABETES___E14,HP,FG: Diabetes mellitus & \nUK: Unspecified diab...,0.420760,0.098911,4.253925,2.100555e-05,2,*
E4_DIABETES___E14,PTB,FG: Diabetes mellitus & \nUK: Unspecified diab...,0.300049,0.316877,0.946894,3.436927e-01,2,
E4_OBESITY___E66,GD,FG: Obesity & \nUK: Obesity,0.529482,0.153193,3.456307,5.476317e-04,2,
E4_OBESITY___E66,HP,FG: Obesity & \nUK: Obesity,0.374283,0.092909,4.028490,5.613619e-05,2,*
E4_OBESITY___E66,PTB,FG: Obesity & \nUK: Obesity,0.175672,0.260471,0.674440,5.000318e-01,2,
I9_HYPTENSESS___I10,GD,"FG: Hypertension, essential & \nUK: Essential ...",0.500904,0.141347,3.543789,3.944202e-04,2,
I9_HYPTENSESS___I10,HP,"FG: Hypertension, essential & \nUK: Essential ...",0.699877,0.101525,6.893642,5.438180e-12,2,*
I9_HYPTENSESS___I10,PTB,"FG: Hypertension, essential & \nUK: Essential ...",0.319135,0.311526,1.024425,3.056346e-01,2,
I9_IHD___I25,GD,"FG: Ischaemic heart disease, wide definition &...",0.247124,0.091702,2.694859,7.041833e-03,2,
